### Setting Up the data

In [1]:
import os
source_language = "en"
target_language = "ln" # ln is the language code of lingala 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# No need to use gdrive since am using a gpu
!mkdir -p "$src-$tgt-$tag"
os.environ["gdrive_path"] = "%s-%s-%s" % (source_language, target_language, tag) # saving directly on the vm

In [2]:
!echo $gdrive_path

en-ln-baseline


#### Downloading the corpus data

In [3]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ln.xml.gz not found. The following files are available for downloading:

   5 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-ln.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  60 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ln.zip

 328 MB Total size
./JW300_latest_xml_en-ln.xml.gz ... 100% of 5 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_ln.zip ... 100% of 60 MB


In [4]:
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-27 07:54:38--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.04s   

2020-01-27 07:54:39 (6.66 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-27 07:54:39--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-ln.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200

In [5]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [6]:
!ls

JW300_latest_xml_en-ln.xml  baseline_2020.ipynb  jw300.ln	 test.ln
JW300_latest_xml_en.zip     en-ln-baseline	 test.en
JW300_latest_xml_ln.zip     jw300.en		 test.en-any.en


#### Building the model dataset

In [7]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(10)

Loaded data and skipped 6663/601113 lines since contained in test set.


,source_sentence,target_sentence
0,Who Wants to Be a Millionaire ?,Nani alingi kozala milionere ?
1,THE answer seems to be : almost everybody .,"EYANO emonani lokola ete , wana ezali mposa ya..."
2,And the easiest way to become one ​ — accordin...,"Nzokande , na makanisi ya bato , nzela ya pɛtɛ..."
3,Pandering to prevailing taste ​ — and wanting ...,Kolamusáká mposa ya bato mingi ​ — mpe kolulák...
4,A few people do become millionaires .,Mwa babɛti na yango bazali mpenza kokóma bamil...
5,One Englishman had filled out soccer coupons f...,Mongelesi moko oyo azalaki kosala momekano na ...
6,"For a stake of 50 cents , he won nearly $ 1.5 ...",Abɛtaki bobele na mosolo mokokani na franka 3 ...
7,Even more spectacular was the payoff for a wom...,Oyo ekamwisi mpenza ezali likambo ya mwasi oyo...
8,But they are exceptions .,Kasi baoyo bazali kolónga boye bazali sé moke ...
9,More typical is the middle - ​ aged Spanish cl...,Ndakisa emonisi yango malamu ezali mosali na b...


Some tests :

In [8]:
df.source_sentence[100]

'And why does money mean so much to him ?'

In [9]:
df.target_sentence[100]

'Mpo na nini mbongo ezali na ndimbola monene mpo na ye ?'

### Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [10]:
df_without_duplicates = df.drop_duplicates()

In [11]:
df_without_duplicates = df_without_duplicates.drop_duplicates(subset='source_sentence', inplace=False)
df_without_duplicates = df_without_duplicates.drop_duplicates(subset='target_sentence', inplace=False)

In [12]:
df_without_duplicates.shape

(539980, 2)

In [14]:
df_without_duplicates = df_without_duplicates.sample(frac=1, random_state=seed).reset_index(drop=True)

In [17]:
! pip3 install fuzzywuzzy
! pip3 install python-Levenshtein

    100% |████████████████████████████████| 51kB 2.1MB/s ta 0:00:011
    100% |████████████████████████████████| 593kB 1.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for python-Levenshtein ... done
  Stored in directory: /home/espoir_mur_gmail_com/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [99]:
import time
from fuzzywuzzy import process
import numpy as np
from multiprocessing import Pool

In [19]:
## reset the index after
df_without_duplicates.reset_index(inplace=True, drop=False)

In [20]:
# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
    candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
    if len(candidates) > 0:
        return process.extractOne(sample, candidates)[1]
    else:
        return np.nan

TODOS : Next step will be to save the result to a text file , and reload it if something bad happens or  write the process in parallel

In [98]:
#df_without_duplicates.source_sentence

0         Shepherds of God’s flock need to give loving a...
1         Claire explained that as one of Jehovah’s Witn...
2         Thus , during the next ten years , in a settin...
3         They pick up a newspaper and read about bomb b...
4         For instance , Christian elders are greatly ne...
                                ...                        
539975    In recent years the United Nations has been st...
539976    Indeed , a good leader practices what he preac...
539977    While they did not escape all the consequences...
539978    It is a shame that King Solomon’s son Rehoboam...
539979    In putting his service to God first , Jesus di...
Name: source_sentence, Length: 537792, dtype: object

In [ ]:
#with Pool(5) as p:
 #   print(p.map(f, [1, 2, 3]))

In [21]:
# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_without_duplicates.iterrows():
    scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
    if idx % 1000 == 0:
        hours, rem = divmod(time.time() - start_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_without_duplicates))))

# Filter out "almost overlapping samples"
df_without_duplicates['scores'] = scores
df_without_duplicates = df_without_duplicates[df_without_duplicates['scores'] < 95]

00:00:00.20 0.00 percent complete
00:00:23.79 0.19 percent complete
00:00:46.52 0.37 percent complete
00:01:11.27 0.56 percent complete
00:01:35.40 0.74 percent complete
00:01:58.11 0.93 percent complete
00:02:22.19 1.11 percent complete
00:02:45.60 1.30 percent complete
00:03:08.70 1.48 percent complete
00:03:32.31 1.67 percent complete
00:03:55.34 1.85 percent complete
00:04:19.00 2.04 percent complete
00:04:41.43 2.22 percent complete
00:05:05.32 2.41 percent complete
00:05:29.59 2.59 percent complete
00:05:53.31 2.78 percent complete
00:06:16.47 2.96 percent complete
00:06:40.38 3.15 percent complete
00:07:04.18 3.33 percent complete
00:07:27.66 3.52 percent complete
00:07:51.61 3.70 percent complete
00:08:15.31 3.89 percent complete
00:08:38.80 4.07 percent complete
00:09:02.71 4.26 percent complete
00:09:26.52 4.44 percent complete
00:09:49.76 4.63 percent complete


00:10:13.14 4.81 percent complete
00:10:36.41 5.00 percent complete
00:11:00.88 5.19 percent complete
00:11:24.70 5.37 percent complete
00:11:48.24 5.56 percent complete
00:12:11.88 5.74 percent complete


00:12:34.99 5.93 percent complete
00:12:58.50 6.11 percent complete
00:13:22.31 6.30 percent complete
00:13:45.75 6.48 percent complete
00:14:10.31 6.67 percent complete
00:14:34.35 6.85 percent complete
00:14:58.29 7.04 percent complete
00:15:21.40 7.22 percent complete
00:15:44.86 7.41 percent complete
00:16:08.42 7.59 percent complete
00:16:32.82 7.78 percent complete
00:16:56.27 7.96 percent complete
00:17:20.00 8.15 percent complete
00:17:43.65 8.33 percent complete
00:18:07.61 8.52 percent complete
00:18:31.07 8.70 percent complete
00:18:54.44 8.89 percent complete
00:19:17.44 9.07 percent complete
00:19:41.24 9.26 percent complete


00:20:04.54 9.44 percent complete
00:20:28.54 9.63 percent complete
00:20:51.76 9.82 percent complete
00:21:14.76 10.00 percent complete
00:21:38.14 10.19 percent complete
00:22:02.40 10.37 percent complete
00:22:26.38 10.56 percent complete
00:22:49.72 10.74 percent complete
00:23:13.24 10.93 percent complete
00:23:37.81 11.11 percent complete
00:24:01.25 11.30 percent complete
00:24:24.46 11.48 percent complete
00:24:47.53 11.67 percent complete
00:25:11.05 11.85 percent complete
00:25:34.05 12.04 percent complete
00:25:57.24 12.22 percent complete
00:26:20.38 12.41 percent complete
00:26:44.74 12.59 percent complete
00:27:08.82 12.78 percent complete
00:27:32.45 12.96 percent complete


00:27:55.82 13.15 percent complete
00:28:19.30 13.33 percent complete
00:28:42.14 13.52 percent complete
00:29:05.13 13.70 percent complete
00:29:27.90 13.89 percent complete
00:29:52.06 14.07 percent complete
00:30:15.07 14.26 percent complete
00:30:39.00 14.44 percent complete
00:31:02.86 14.63 percent complete
00:31:26.43 14.82 percent complete
00:31:50.26 15.00 percent complete
00:32:13.16 15.19 percent complete
00:32:36.98 15.37 percent complete
00:33:01.38 15.56 percent complete


...

02:19:49.72 65.93 percent complete
02:20:13.15 66.11 percent complete
02:20:36.74 66.30 percent complete
02:21:00.40 66.48 percent complete
02:21:24.24 66.67 percent complete
02:21:47.71 66.85 percent complete
02:22:11.50 67.04 percent complete
02:22:34.83 67.22 percent complete
02:22:58.59 67.41 percent complete
02:23:22.27 67.60 percent complete
02:23:45.44 67.78 percent complete
02:24:09.40 67.97 percent complete
02:24:33.37 68.15 percent complete
02:24:56.79 68.34 percent complete
02:25:20.33 68.52 percent complete
02:25:43.62 68.71 percent complete
02:26:07.90 68.89 percent complete
02:26:31.04 69.08 percent complete
02:26:54.16 69.26 percent complete
02:27:17.65 69.45 percent complete
02:27:41.45 69.63 percent complete
02:28:04.83 69.82 percent complete


02:28:28.00 70.00 percent complete
02:28:51.37 70.19 percent complete
02:29:14.45 70.37 percent complete
02:29:38.45 70.56 percent complete
02:30:01.48 70.74 percent complete
02:30:25.48 70.93 percent complete
02:30:49.06 71.11 percent complete
02:31:12.21 71.30 percent complete
02:31:35.63 71.48 percent complete
02:31:59.32 71.67 percent complete
02:32:22.52 71.85 percent complete
02:32:45.58 72.04 percent complete
02:33:09.23 72.22 percent complete
02:33:32.88 72.41 percent complete
02:33:56.73 72.60 percent complete
02:34:20.52 72.78 percent complete
02:34:43.54 72.97 percent complete
02:35:07.11 73.15 percent complete
02:35:30.17 73.34 percent complete
02:35:53.57 73.52 percent complete
02:36:17.02 73.71 percent complete
02:36:40.56 73.89 percent complete
02:37:05.02 74.08 percent complete
02:37:28.07 74.26 percent complete
02:37:51.47 74.45 percent complete
02:38:14.72 74.63 percent complete
02:38:38.89 74.82 percent complete
02:39:02.56 75.00 percent complete
02:39:26.20 75.19 pe

03:12:04.94 90.56 percent complete
03:12:28.12 90.74 percent complete


03:12:52.12 90.93 percent complete
03:13:15.39 91.11 percent complete
03:13:38.56 91.30 percent complete
03:14:01.81 91.48 percent complete
03:14:25.25 91.67 percent complete
03:14:48.06 91.86 percent complete
03:15:12.70 92.04 percent complete
03:15:36.48 92.23 percent complete
03:15:59.39 92.41 percent complete
03:16:23.20 92.60 percent complete
03:16:46.70 92.78 percent complete
03:17:09.74 92.97 percent complete
03:17:33.37 93.15 percent complete
03:17:56.43 93.34 percent complete
03:18:20.71 93.52 percent complete
03:18:44.10 93.71 percent complete


03:19:07.37 93.89 percent complete
03:19:30.39 94.08 percent complete
03:19:54.47 94.26 percent complete
03:20:17.90 94.45 percent complete
03:20:41.64 94.63 percent complete
03:21:05.26 94.82 percent complete
03:21:28.74 95.00 percent complete
03:21:52.00 95.19 percent complete
03:22:15.78 95.37 percent complete
03:22:38.97 95.56 percent complete
03:23:01.92 95.74 percent complete
03:23:24.74 95.93 percent complete
03:23:48.57 96.11 percent complete
03:24:12.84 96.30 percent complete
03:24:36.87 96.49 percent complete
03:25:00.31 96.67 percent complete
03:25:24.52 96.86 percent complete
03:25:48.10 97.04 percent complete
03:26:11.18 97.23 percent complete
03:26:34.13 97.41 percent complete
03:26:58.46 97.60 percent complete
03:27:21.91 97.78 percent complete
03:27:46.15 97.97 percent complete
03:28:09.84 98.15 percent complete
03:28:33.46 98.34 percent complete
03:28:56.39 98.52 percent complete
03:29:19.47 98.71 percent complete
03:29:43.16 98.89 percent complete
03:30:06.23 99.08 pe

In [25]:
df_without_duplicates.to_csv('data_process_without_duplicates.csv')

splitting the data into train/test sets

In [28]:

# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_without_duplicates["source_sentence"] = df_without_duplicates["source_sentence"].str.lower()
    df_without_duplicates["target_sentence"] = df_without_duplicates["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_without_duplicates.tail(num_dev_patterns) # Herman: Error in original
stripped = df_without_duplicates.drop(df_without_duplicates.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
    for index, row in stripped.iterrows():
        src_file.write(row["source_sentence"]+"\n")
        trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
    for index, row in dev.iterrows():
        src_file.write(row["source_sentence"]+"\n")
        trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
Shepherds of God’s flock need to give loving assistance to Christians who have erred but are truly repentant .
Claire explained that as one of Jehovah’s Witnesses , she did not casually date boys .
Thus , during the next ten years , in a setting of magnificent mountains , forests , and lakes in northern Sumatra , we worked with missionaries from Australia , Austria , Germany , the Philippines , Sweden , and the United States .
They pick up a newspaper and read about bomb blasts that rip through buildings , killing scores of innocent people .
For instance , Christian elders are greatly needed .
( June 22 , 2002 ) It was exciting to read about Sigismund , who defended the anti - Trinitarians in their debates , and about how legal recognition was given to the Unitarian Church .
He recalls : “ Our brief conversation was enough to give me renewed courage .
Read Psalms 148 - 150 , and observe how often these three psalms encourage us to praise Jehovah .
You might even have r

In [32]:
# checking if joeymnt is installed and skip the installation 

In [31]:
!pip3 freeze | grep joeynmt

joeynmt==0.0.1


In [33]:
!pip3 freeze | grep subword-nmt

subword-nmt==0.3.6


In [34]:
!which subword-nmt

/usr/local/bin/subword-nmt


### Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization (Sennrich, 2015) .

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages (Sennrich, 2019) (Martinus, 2019)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by (Sennrich, 2019). You do not need to change anything. Simply running the below will be suitable.

In [35]:
from os import path

In [36]:
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

create the new data folder from the working directory since joeymnt is already installed we are using another data folder

In [38]:
!mkdir data

In [37]:
# Learn BPEs on the training data.
os.environ["data_path"] = path.join("data", source_language + target_language) # Herma

In [39]:
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt


In [40]:
# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

creating the data folder from my language and copying everything to it

In [41]:
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

bpe.codes.4000	dev.en	     test.bpe.ln     test.ln	   train.en
dev.bpe.en	dev.ln	     test.en	     train.bpe.en  train.ln
dev.bpe.ln	test.bpe.en  test.en-any.en  train.bpe.ln


Creating the vocabulary using joeymnt 

getting the script manually and checking  and run it

In [55]:
!wget https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py

--2020-01-27 18:37:20--  https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2034 (2.0K) [text/plain]
Saving to: ‘build_vocab.py’

build_vocab.py      100%[===================>]   1.99K  --.-KB/s    in 0s      

2020-01-27 18:37:20 (39.3 MB/s) - ‘build_vocab.py’ saved [2034/2034]



In [59]:
!python3 build_vocab.py data/$src$tgt/train.bpe.$src data/$src$tgt/train.bpe.$tgt --output_path data/$src$tgt/vocab.txt

In [60]:
# Some output
! echo "BPE Lingala Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 data/$src$tgt/vocab.txt 

BPE Lingala Sentences
Yango wana , ezali na ntina mingi ete baboti bá@@ koba koteya bana na bango . ”
Tiká bana na yo bá@@ mona ete Yehova azali mpenza solo na miso na yo .
B@@ alobi boye : “ T@@ oy@@ ebisaki mpe mwana na biso ya mwasi ya liboso ‘ at@@ y@@ ela Yehova motema mobimba , ak@@ oba komipesa na mosala ya Bokonzi , mpe am@@ itungisa mingi te . ’
Ntango amonaki ndenge likambo yango es@@ ukaki , ayebaki ete Yehova azalaki kosalisa biso .
Yango ekóm@@ isaki makasi kondima na ye epai ya Nzambe mpe na Biblia . ”
Combined BPE Vocab
ò
✘
κ@@
ά@@
ʺ
(@@
Ṭ@@
ʽ
ă
ingomba


### Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture
- We set our dropout to reasonably high: 0.3 (recommended in (Sennrich, 2019))
Things worth playing with:

- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
Evaluation metrics (BLEU versus Crhf4)

In [62]:
!ls 

JW300_latest_xml_en-ln.xml	     dev.bpe.ln      test.en-any.en
JW300_latest_xml_en.zip		     dev.en	     test.ln
JW300_latest_xml_ln.zip		     dev.ln	     train.bpe.en
baseline_2020.ipynb		     en-ln-baseline  train.bpe.ln
bpe.codes.4000			     jw300.en	     train.en
build_vocab.py			     jw300.ln	     train.ln
data				     test.bpe.en     vocab.en
data_process_without_duplicates.csv  test.bpe.ln     vocab.ln
dev.bpe.en			     test.en


In [63]:
!mkdir models

In [64]:
!mkdir config

In [66]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '{}{}'.format(source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("config/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [67]:
!ls config

transformer_enln.yaml


### Train the Model

Trying to fix the issue with tensorboard not found

In [87]:
# !pip3 freeze | grep tensorboard

tensorboard==1.14.0


In [86]:
# !pip3 install tensorboard==1.14.0

    100% |████████████████████████████████| 3.2MB 448kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/a7/c5/6c1acea1b4ea88b86b03280f3fde1efa04fefecd4e7d2af13e602661cde4/setuptools-45.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ca/ac/838c8c8a5f33a58132dd2ad2a30329f6ae1614a9f56ffb79eaaf71a9d156/protobuf-3.11.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c2/e4/a859d2fe516f466642fa5c6054fd9646271f9da26b0cac0d2f37fc858c8f/Werkzeug-0.16.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c0/4e/fd492e91abdc2d2fcb70ef453064d980688762079397f779758e055f6575/Markdown-3.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8b/9b/ba5d094f979325fdba696bafa9ee23cc50b8fc60481e3d2a9e13d768

In [83]:
!pip3 install --upgrade torch

  Using cached https://files.pythonhosted.org/packages/24/19/4804aea17cd136f1705a5e98a00618cb8f6ccc375ad8bfa437408e09d058/torch-1.4.0-cp36-cp36m-manylinux1_x86_64.whl


This single line of joeynmt runs the training using the config we made above

In [91]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!python3 -m joeynmt train config/transformer_$src$tgt.yaml

/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/espoir_mur_gmail_com/.

2020-01-27 19:55:34,396 cfg.name                           : enln_transformer
2020-01-27 19:55:34,397 cfg.data.src                       : en
2020-01-27 19:55:34,397 cfg.data.trg                       : ln
2020-01-27 19:55:34,397 cfg.data.train                     : data/enln/train.bpe
2020-01-27 19:55:34,397 cfg.data.dev                       : data/enln/dev.bpe
2020-01-27 19:55:34,397 cfg.data.test                      : data/enln/test.bpe
2020-01-27 19:55:34,397 cfg.data.level                     : bpe
2020-01-27 19:55:34,397 cfg.data.lowercase                 : False
2020-01-27 19:55:34,398 cfg.data.max_sent_length           : 100
2020-01-27 19:55:34,398 cfg.data.src_vocab                 : data/enln/vocab.txt
2020-01-27 19:55:34,398 cfg.data.trg_vocab                 : data/enln/vocab.txt
2020-01-27 19:55:34,398 cfg.testing.beam_size              : 5
2020-01-27 19:55:34,398 cfg.testing.alpha                  : 1.0
2020-01-27 19:55:34,398 cfg.training.random_seed           : 42
202

2020-01-27 20:02:36,879 Epoch   1 Step:     1100 Batch Loss:     3.740876 Tokens per Sec:     7796, Lr: 0.000300
2020-01-27 20:03:06,989 Epoch   1 Step:     1200 Batch Loss:     3.734322 Tokens per Sec:     7673, Lr: 0.000300
2020-01-27 20:03:37,312 Epoch   1 Step:     1300 Batch Loss:     3.573833 Tokens per Sec:     7565, Lr: 0.000300
2020-01-27 20:04:07,388 Epoch   1 Step:     1400 Batch Loss:     3.300878 Tokens per Sec:     7676, Lr: 0.000300
2020-01-27 20:04:37,703 Epoch   1 Step:     1500 Batch Loss:     3.694288 Tokens per Sec:     7727, Lr: 0.000300
2020-01-27 20:05:08,044 Epoch   1 Step:     1600 Batch Loss:     3.443311 Tokens per Sec:     7683, Lr: 0.000300
2020-01-27 20:05:37,961 Epoch   1 Step:     1700 Batch Loss:     3.393305 Tokens per Sec:     7670, Lr: 0.000300
2020-01-27 20:06:08,197 Epoch   1 Step:     1800 Batch Loss:     3.223591 Tokens per Sec:     7724, Lr: 0.000300
2020-01-27 20:06:38,635 Epoch   1 Step:     1900 Batch Loss:     3.097262 Tokens per Sec:     78

2020-01-27 20:22:10,068 Epoch   1 Step:     4100 Batch Loss:     2.793284 Tokens per Sec:     7692, Lr: 0.000300
2020-01-27 20:22:40,278 Epoch   1 Step:     4200 Batch Loss:     3.477712 Tokens per Sec:     7722, Lr: 0.000300
2020-01-27 20:23:10,649 Epoch   1 Step:     4300 Batch Loss:     2.809034 Tokens per Sec:     7672, Lr: 0.000300
2020-01-27 20:23:40,356 Epoch   1 Step:     4400 Batch Loss:     2.726703 Tokens per Sec:     7596, Lr: 0.000300
2020-01-27 20:24:10,892 Epoch   1 Step:     4500 Batch Loss:     2.718876 Tokens per Sec:     7750, Lr: 0.000300
2020-01-27 20:24:41,138 Epoch   1 Step:     4600 Batch Loss:     2.652498 Tokens per Sec:     7727, Lr: 0.000300
2020-01-27 20:25:11,695 Epoch   1 Step:     4700 Batch Loss:     2.736690 Tokens per Sec:     7800, Lr: 0.000300
2020-01-27 20:25:41,860 Epoch   1 Step:     4800 Batch Loss:     2.644743 Tokens per Sec:     7634, Lr: 0.000300
2020-01-27 20:26:41,972 Epoch   1 Step:     5000 Batch Loss:     3.278113 Tokens per Sec:     76

2020-01-27 20:41:44,759 Epoch   2 Step:     7100 Batch Loss:     2.317835 Tokens per Sec:     7695, Lr: 0.000300
2020-01-27 20:42:15,082 Epoch   2 Step:     7200 Batch Loss:     2.546905 Tokens per Sec:     7651, Lr: 0.000300
2020-01-27 20:42:45,091 Epoch   2 Step:     7300 Batch Loss:     2.100610 Tokens per Sec:     7663, Lr: 0.000300
2020-01-27 20:43:15,167 Epoch   2 Step:     7400 Batch Loss:     2.012168 Tokens per Sec:     7686, Lr: 0.000300
2020-01-27 20:43:44,790 Epoch   2 Step:     7500 Batch Loss:     2.024819 Tokens per Sec:     7479, Lr: 0.000300
2020-01-27 20:44:15,067 Epoch   2 Step:     7600 Batch Loss:     2.246175 Tokens per Sec:     7697, Lr: 0.000300
2020-01-27 20:44:45,000 Epoch   2 Step:     7700 Batch Loss:     2.431381 Tokens per Sec:     7663, Lr: 0.000300
2020-01-27 20:45:15,097 Epoch   2 Step:     7800 Batch Loss:     2.152147 Tokens per Sec:     7531, Lr: 0.000300
2020-01-27 20:45:45,122 Epoch   2 Step:     7900 Batch Loss:     2.259594 Tokens per Sec:     76

...

2020-01-28 14:12:38,167 Epoch  28 Step:   166100 Batch Loss:     1.158849 Tokens per Sec:     4162, Lr: 0.000300
2020-01-28 14:13:33,366 Epoch  28 Step:   166200 Batch Loss:     1.162447 Tokens per Sec:     4120, Lr: 0.000300
2020-01-28 14:14:28,858 Epoch  28 Step:   166300 Batch Loss:     1.389554 Tokens per Sec:     4190, Lr: 0.000300
2020-01-28 14:15:24,174 Epoch  28 Step:   166400 Batch Loss:     1.381477 Tokens per Sec:     4220, Lr: 0.000300
2020-01-28 14:16:20,300 Epoch  28 Step:   166500 Batch Loss:     1.314907 Tokens per Sec:     4164, Lr: 0.000300
2020-01-28 14:17:16,101 Epoch  28 Step:   166600 Batch Loss:     1.738812 Tokens per Sec:     4146, Lr: 0.000300
2020-01-28 14:18:11,814 Epoch  28 Step:   166700 Batch Loss:     1.580838 Tokens per Sec:     4191, Lr: 0.000300
2020-01-28 14:19:06,673 Epoch  28 Step:   166800 Batch Loss:     1.516567 Tokens per Sec:     4203, Lr: 0.000300
2020-01-28 14:20:02,273 Epoch  28 Step:   166900 Batch Loss:     1.218654 Tokens per Sec:     42

2020-01-28 14:49:15,210 Epoch  29 Step:   169100 Batch Loss:     1.193205 Tokens per Sec:     4128, Lr: 0.000300
2020-01-28 14:50:10,139 Epoch  29 Step:   169200 Batch Loss:     1.277048 Tokens per Sec:     4202, Lr: 0.000300
2020-01-28 14:51:04,617 Epoch  29 Step:   169300 Batch Loss:     1.466521 Tokens per Sec:     4101, Lr: 0.000300
2020-01-28 14:52:00,120 Epoch  29 Step:   169400 Batch Loss:     1.222802 Tokens per Sec:     4244, Lr: 0.000300
2020-01-28 14:52:55,148 Epoch  29 Step:   169500 Batch Loss:     1.298234 Tokens per Sec:     4148, Lr: 0.000300
2020-01-28 14:53:50,532 Epoch  29 Step:   169600 Batch Loss:     1.610779 Tokens per Sec:     4191, Lr: 0.000300
2020-01-28 14:54:46,836 Epoch  29 Step:   169700 Batch Loss:     1.412869 Tokens per Sec:     4170, Lr: 0.000300
2020-01-28 14:55:43,014 Epoch  29 Step:   169800 Batch Loss:     1.547729 Tokens per Sec:     4256, Lr: 0.000300
2020-01-28 14:56:38,596 Epoch  29 Step:   169900 Batch Loss:     1.528722 Tokens per Sec:     40

2020-01-28 15:25:48,457 Epoch  29 Step:   172100 Batch Loss:     1.208136 Tokens per Sec:     4220, Lr: 0.000300
2020-01-28 15:26:43,812 Epoch  29 Step:   172200 Batch Loss:     1.477997 Tokens per Sec:     4205, Lr: 0.000300
2020-01-28 15:27:39,190 Epoch  29 Step:   172300 Batch Loss:     1.410105 Tokens per Sec:     4274, Lr: 0.000300
2020-01-28 15:28:34,911 Epoch  29 Step:   172400 Batch Loss:     1.436419 Tokens per Sec:     4206, Lr: 0.000300
2020-01-28 15:29:30,143 Epoch  29 Step:   172500 Batch Loss:     1.281025 Tokens per Sec:     4234, Lr: 0.000300
2020-01-28 15:30:25,389 Epoch  29 Step:   172600 Batch Loss:     1.377638 Tokens per Sec:     4296, Lr: 0.000300
2020-01-28 15:31:20,502 Epoch  29 Step:   172700 Batch Loss:     1.213027 Tokens per Sec:     4179, Lr: 0.000300
2020-01-28 15:32:16,492 Epoch  29 Step:   172800 Batch Loss:     1.542442 Tokens per Sec:     4169, Lr: 0.000300
2020-01-28 15:33:12,361 Epoch  29 Step:   172900 Batch Loss:     1.150009 Tokens per Sec:     42

2020-01-28 16:02:20,526 Epoch  30 Step:   175100 Batch Loss:     1.283347 Tokens per Sec:     4170, Lr: 0.000300
2020-01-28 16:03:15,944 Epoch  30 Step:   175200 Batch Loss:     1.366150 Tokens per Sec:     4188, Lr: 0.000300
2020-01-28 16:04:10,899 Epoch  30 Step:   175300 Batch Loss:     1.386524 Tokens per Sec:     4263, Lr: 0.000300
2020-01-28 16:05:06,385 Epoch  30 Step:   175400 Batch Loss:     1.477812 Tokens per Sec:     4177, Lr: 0.000300
2020-01-28 16:06:01,889 Epoch  30 Step:   175500 Batch Loss:     1.555101 Tokens per Sec:     4187, Lr: 0.000300
2020-01-28 16:06:57,734 Epoch  30 Step:   175600 Batch Loss:     1.294442 Tokens per Sec:     4253, Lr: 0.000300
2020-01-28 16:07:53,088 Epoch  30 Step:   175700 Batch Loss:     1.231424 Tokens per Sec:     4151, Lr: 0.000300
2020-01-28 16:08:48,065 Epoch  30 Step:   175800 Batch Loss:     1.223286 Tokens per Sec:     4161, Lr: 0.000300
2020-01-28 16:09:44,039 Epoch  30 Step:   175900 Batch Loss:     1.495184 Tokens per Sec:     41

2020-01-28 16:38:56,403 Epoch  30 Step:   178100 Batch Loss:     1.317670 Tokens per Sec:     4163, Lr: 0.000300
2020-01-28 16:39:51,001 Epoch  30 Step:   178200 Batch Loss:     1.734640 Tokens per Sec:     4111, Lr: 0.000300
2020-01-28 16:40:46,770 Epoch  30 Step:   178300 Batch Loss:     1.117447 Tokens per Sec:     4223, Lr: 0.000300
2020-01-28 16:41:41,745 Epoch  30 Step:   178400 Batch Loss:     1.184216 Tokens per Sec:     4199, Lr: 0.000300
2020-01-28 16:42:36,664 Epoch  30 Step:   178500 Batch Loss:     1.240675 Tokens per Sec:     4173, Lr: 0.000300
2020-01-28 16:43:32,213 Epoch  30 Step:   178600 Batch Loss:     1.300894 Tokens per Sec:     4225, Lr: 0.000300
2020-01-28 16:44:26,906 Epoch  30 Step:   178700 Batch Loss:     1.330272 Tokens per Sec:     4163, Lr: 0.000300
2020-01-28 16:45:22,409 Epoch  30 Step:   178800 Batch Loss:     1.439127 Tokens per Sec:     4267, Lr: 0.000300
2020-01-28 16:46:17,519 Epoch  30 Step:   178900 Batch Loss:     1.311184 Tokens per Sec:     41

In [92]:
!ls models/${src}${tgt}_transformer

00178000.hyps.dev   132000.hyps  168000.hyps  38000.hyps  73000.hyps
00178000.hyps.test  133000.hyps  169000.hyps  39000.hyps  74000.hyps
1000.hyps	    134000.hyps  17000.hyps   4000.hyps   75000.hyps
10000.hyps	    135000.hyps  170000.hyps  40000.hyps  76000.hyps
100000.hyps	    136000.hyps  171000.hyps  41000.hyps  77000.hyps
101000.hyps	    137000.hyps  172000.hyps  42000.hyps  78000.hyps
102000.hyps	    138000.hyps  173000.ckpt  43000.hyps  79000.hyps
103000.hyps	    139000.hyps  173000.hyps  44000.hyps  8000.hyps
104000.hyps	    14000.hyps	 174000.ckpt  45000.hyps  80000.hyps
105000.hyps	    140000.hyps  174000.hyps  46000.hyps  81000.hyps
106000.hyps	    141000.hyps  175000.hyps  47000.hyps  82000.hyps
107000.hyps	    142000.hyps  176000.hyps  48000.hyps  83000.hyps
108000.hyps	    143000.hyps  177000.hyps  49000.hyps  84000.hyps
109000.hyps	    144000.hyps  178000.ckpt  5000.hyps   85000.hyps
11000.hyps	    145000.hyps  178000.hyps  50000.hyps  86000.hyps
110000.hyps	    146000.

In [96]:
! cat models/${src}${tgt}_transformer/validations.txt

Steps: 1000	Loss: 98587.16406	PPL: 42.35123	bleu: 1.97804	LR: 0.00030000	*
Steps: 2000	Loss: 83331.87500	PPL: 23.72063	bleu: 4.41728	LR: 0.00030000	*
Steps: 3000	Loss: 74468.50781	PPL: 16.93810	bleu: 6.43743	LR: 0.00030000	*
Steps: 4000	Loss: 68886.13281	PPL: 13.70078	bleu: 9.43805	LR: 0.00030000	*
Steps: 5000	Loss: 64822.57422	PPL: 11.74058	bleu: 11.82589	LR: 0.00030000	*
Steps: 6000	Loss: 61739.46484	PPL: 10.44269	bleu: 13.63813	LR: 0.00030000	*
Steps: 7000	Loss: 59067.85547	PPL: 9.43466	bleu: 14.85649	LR: 0.00030000	*
Steps: 8000	Loss: 56806.56250	PPL: 8.65786	bleu: 16.22938	LR: 0.00030000	*
Steps: 9000	Loss: 54457.80078	PPL: 7.91867	bleu: 18.71813	LR: 0.00030000	*
Steps: 10000	Loss: 52996.16406	PPL: 7.49087	bleu: 19.07842	LR: 0.00030000	*
Steps: 11000	Loss: 51604.36719	PPL: 7.10502	bleu: 19.20304	LR: 0.00030000	*
Steps: 12000	Loss: 50352.13672	PPL: 6.77487	bleu: 20.38498	LR: 0.00030000	*
Steps: 13000	Loss: 49454.42188	PPL: 6.54768	bleu: 20.43895	LR: 0.00030000	*
Steps: 14000	Loss: 

In [97]:
!python3 -m joeynmt test config/transformer_$src$tgt.yaml

/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/espoir_mur_gmail_com/.